# Deteksi Outlier ABOD 

## Angle-Based Outlier Detection (ABOD)  
Angle-Based Outlier Detection (ABOD) adalah pendekatan geometris untuk mengidentifikasi outlier dalam data berdimensi tinggi dengan menganalisis varians sudut yang dibentuk oleh vektor-vektor dari suatu titik ke tetangganya.

Dalam satu himpunan data, titik-titik normal cenderung memiliki sudut-sudut yang serupa antara pasangan vektor yang menghubungkannya ke titik lain. Sebaliknya, outlier sering kali terletak di tepi dan membentuk rentang sudut yang sangat luas terhadap tetangganya — artinya, varians sudut mereka tinggi.

## Algoritma Angle-Based Outlier Detection (ABOD)

1. Untuk setiap titik \( p_i \) dalam dataset:
   - Hitung semua vektor pasangan dari \( p_i \) ke titik lainnya:  
     $$
     \vec{v}_{ij} = p_j - p_i
     $$
   - Untuk setiap pasangan vektor seperti $((\vec{v}_{ij}, \vec{v}_{ik})$ , hitung sudut di antara keduanya menggunakan rumus hasil kali titik:  
     $$
     \cos(\theta_{ijk}) = \frac{\vec{v}_{ij} \cdot \vec{v}_{ik}}{\|\vec{v}_{ij}\| \cdot \|\vec{v}_{ik}\|}
     $$
   - Hitung **varians** dari semua sudut tersebut untuk titik \( p_i \).


2. Skor outlier untuk titik $ p_i $ adalah varians sudut:  
   $$
   ABOD(p_i) = \mathrm{Var}(\theta_{ijk}) \quad \text{untuk semua } j, k \neq i
   $$


3. Titik-titik dengan **varians sudut tinggi** ditandai sebagai **outlier**.


## Implementasi
### Import library yang dibutuhkan 

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

### Load dataset

In [2]:
load_dotenv()

# Ambil variabel dari .env
user_postgres = os.getenv("USER_POSTGRES")
password_postgres = os.getenv("PASS_POSTGRES")
host_postgres = os.getenv("HOST_POSTGRES")
port_postgres = os.getenv("PORT_POSTGRES")
db_postgres = os.getenv("DB_POSTGRES")

user_mysql = os.getenv("USER_MYSQL")
password_mysql = os.getenv("PASS_MYSQL")
host_mysql = os.getenv("HOST_MYSQL")
port_mysql = os.getenv("PORT_MYSQL")
db_mysql = os.getenv("DB_MYSQL")


postgres_conn = f"postgresql+psycopg2://{user_postgres}:{password_postgres}@{host_postgres}:{port_postgres}/{db_postgres}"

mysql_conn = f"mysql+pymysql://{user_mysql}:{password_mysql}@{host_mysql}:{port_mysql}/{db_mysql}"

postgres_engine = create_engine(postgres_conn)
postgres_engine = create_engine(mysql_conn)

### Membuat koneksi (engine) dan mengambil dari database

In [3]:
# Engine untuk PostgreSQL
postgres_engine = create_engine(postgres_conn)

# Engine untuk MySQL
mysql_engine = create_engine(mysql_conn)


# Ambil data dari MySQL
mysql_df_iris = pd.read_sql("SELECT * FROM iris_full", mysql_engine)

# Ambil data dari PostgreSQL
postgres_df_iris = pd.read_sql("SELECT * FROM iris_full", postgres_engine)


### Mengabungkan 2 database

In [4]:

merge_df = pd.merge(mysql_df_iris, postgres_df_iris, left_on="id", right_on='id', how='outer')
print(merge_df)

      id  petal length_x  petal width_x Class_x sepal length_x sepal width_x  \
0      1             NaN            NaN     NaN            NaN           NaN   
1      1             NaN            NaN     NaN            NaN           NaN   
2      2             NaN            NaN     NaN            NaN           NaN   
3      2             NaN            NaN     NaN            NaN           NaN   
4      3             NaN            NaN     NaN            NaN           NaN   
..   ...             ...            ...     ...            ...           ...   
295  148             5.2            2.0    None           None          None   
296  149             5.4            2.3    None           None          None   
297  149             5.4            2.3    None           None          None   
298  150             5.1            1.8    None           None          None   
299  150             5.1            1.8    None           None          None   

     sepal length_y  sepal width_y     

### Salinan data
Buat salinan data dari hasil penggabungan 

In [5]:
data_train = merge_df.copy()

### Model ABOD

In [6]:
import pandas as pd
from pycaret.anomaly import *

# === 1. Pastikan dataset sudah ada ===
print("Jumlah baris:", data_train.shape[0])
print("Jumlah kolom:", data_train.shape[1])
print("Kolom dataset:", list(data_train.columns))
print(data_train.head())

# === 2. Pilih hanya kolom numerik (biar PyCaret tidak error) ===
fitur = data_train.select_dtypes(include=['int64', 'float64'])
print("\nKolom numerik yang dipakai untuk anomaly detection:", list(fitur.columns))

# === 3. Setup PyCaret Anomaly Detection (versi terbaru) ===
from pycaret.anomaly import setup, create_model, assign_model
exp_ano = setup(
    data=fitur,
    session_id=42
)

# === 4. Buat model ABOD ===
abod_model = create_model('abod')

# === 5. Assign label anomaly ===
dataset_outliers = assign_model(abod_model)

# === 6. Hapus baris outlier (Anomaly == 1 artinya normal, -1 artinya outlier) ===
dataset_clean = dataset_outliers[dataset_outliers['Anomaly'] == 1]

print("\nJumlah data awal:", len(dataset_outliers))
print("Jumlah data setelah outlier dihapus:", len(dataset_clean))
dataset_clean.head()


ModuleNotFoundError: No module named 'pycaret'

### Menghapus data dengan skor outlier tertinggi

In [25]:
# Urutkan dataset_outliers berdasarkan skor outlier (descending)
outliers_sorted = dataset_outliers.sort_values(by="Anomaly_Score", ascending=False)

# Ambil index 2 skor outlier tertinggi
top2_outliers_index = outliers_sorted.head(2).index

# Hapus 2 data tersebut dari dataset_outliers
dataset_clean_top2 = dataset_outliers.drop(index=top2_outliers_index)

print("Data setelah 2 outlier tertinggi dihapus:")
print(dataset_clean_top2.head())


Data setelah 2 outlier tertinggi dihapus:
   id  petal length_x  petal width_x  sepal length_y  sepal width_y  \
0  36             1.2            0.2             5.0            3.2   
1  36             1.2            0.2             5.0            3.2   
2  37             1.3            0.2             5.5            3.5   
3  37             1.3            0.2             5.5            3.5   
4  38             1.5            0.1             4.9            3.1   

   petal length_y  petal width_y  Anomaly  Anomaly_Score  
0             NaN            NaN        0      -0.011465  
1             1.2            0.2        0      -0.010103  
2             NaN            NaN        0      -0.025892  
3             1.3            0.2        0      -0.024060  
4             NaN            NaN        0      -0.082164  
